# Expected correlations/uncertainties
It is worth calculation expected correlations - both as they are interesting in themselves, and because I (thought I) need(ed) to calculate the analytical Hessian anyway, to improve high stats toys, where numerical estimates of the Hessian run into trouble). (This was resolved by turning to `autograd` as discussed below).

We minimize the log likelihood function

$$
-\log \mathcal L \simeq \sum_i \frac {(N_i - \hat N_i(\theta))^2}{2N_i} 
$$

where $\theta$ are the parameters we try to estimate (which, depending on setup can be $\{x_\pm, y_\pm\}$ or $\{x_\pm, y_\pm, F_i\}$ (for now ignoring that we are actually using a Poissonian likelihood in the full fit). The yields are, as usual

$$ 
\hat N^+_i = h^+\frac{Y^+_i}{\sum_k Y^+_k}\quad,\quad Y_i^+=(F_{-i}+(x_+^2+y_+^2)F_i + 2\sqrt{F_iF_{-i}}(c_i x_+-s_i y_+)
$$
$$
\hat N^-_i = h^-\frac{Y^-_i}{\sum_k Y^-_k}\quad,\quad Y_i^-=(F_{i}+(x_-^2+y_-^2)F_{-i} + 2\sqrt{F_iF_{-i}}(c_i x_-+s_i y_-)
$$


the covariance matrix $V$ can be estimated from the inverse of the Hessian matrix at the minimum

$$\hat V = \left( \frac{\partial^2 (-\log \mathcal L)}{\partial \hat\theta_i\partial\hat\theta_j   }\right)^{-1} $$

(where a **matrix inverse** is taken). For our specific likelihood function, we find

$$ 
\frac{\partial^2 (-\log \mathcal L)}{\partial \theta_i\partial\theta_j   }
=
\sum_i
\frac {\hat N_i(\theta)-N_i}{N_i} \frac{\partial^2 \hat N_i(\theta)}{\partial \theta_i\partial\theta_j}
+
\frac{1}{N_i}
\left(\frac{\partial \hat N_i(\theta)}{\partial \theta_i}\right)
\left(\frac{\partial \hat N_i(\theta)}{\partial \theta_j}\right)
$$

We can simplify this, by noting that we are looking for the infinite statistics case, in which case the first term in negligible, as it scales as $\sqrt{N}$ whereas the latter term scales as $N$. Furthermore,  on average $N_i=\hat N_i(\theta)$, so we are looking for

$$
\mathbb{E}\hat V = \left( 
\sum_i
\frac{1}{\hat N_i(\theta)}
\left(\frac{\partial \hat N_i(\theta)}{\partial \theta_i}\right)
\left(\frac{\partial \hat N_i(\theta)}{\partial \theta_j}\right)
\right)^{-1} $$

We note that
$$
\frac{\partial N_i^\pm}{\partial \theta_j} = \frac{h^\pm}{\sum_k Y^\pm_k}\left(\frac{\partial Y^\pm_i}{\partial \theta_j}-\frac{Y^\pm_i}{\sum_\ell Y^\pm_\ell}\sum_m\frac{\partial Y^\pm_m}{\partial \theta_j}\right).
$$
For the CP-parameters:
$$
\frac{\partial Y^\pm_i}{\partial x_\pm} = 2 h^\pm x_\pm F_{\pm i} + 2h^\pm \sqrt{F_i F_{-i}}c_i
$$
$$
\frac{\partial Y^\pm_i}{\partial y_\pm} = 2 h^\pm y_\pm F_{\pm i} \mp 2h^\pm \sqrt{F_i F_{-i}}s_i
$$
$$
\frac{\partial Y^\mp_i}{\partial x_\pm} = \frac{\partial Y^\mp_i}{\partial y_\pm} = 0.
$$
For the $F_i$ parameters
$$
\frac{\partial Y^+_i }{\partial F_i} = h^+\left(x_+^2 + y_+^2 + \frac{F_{-i}(c_i x_+ - s_i y_+)}{\sqrt{F_i F_{-i}}}\right)
$$
$$
\frac{\partial Y^+_{-i} }{\partial F_{i}} = h^+\left(1 + \frac{F_{-i}(c_{-i} x_+ - s_{-i} y_+)}{\sqrt{F_i F_{-i}}}\right)
$$
$$
\frac{\partial Y^-_{-i} }{\partial F_{i}} = h^-\left(x_-^2 + y_-^2 + \frac{F_{-i}(c_{-i} x_- + s_{-i} y_-)}{\sqrt{F_i F_{-i}}}\right)
$$
$$
\frac{\partial Y^-_i }{\partial F_{i}} = h^-\left(1 + \frac{F_{-i}(c_i x_- + s_i y_-)}{\sqrt{F_i F_{-i}}}\right)
$$
$$
\frac{\partial Y^\pm_i}{\partial F_j} = 0 \text{     for } |j|\neq |i|
$$
In these calculations, we will pretend the fit is made with the final $F_N$ fixed, which removes the arbitrary normalisation in the equations (if this is not done, the obtained Hession cannot be inverted). 

For us in the calculations below, we use a DefaultModel.py and load $F_i, c_i, s_i$ corresponding to either KsPiPi or KsKK. Here the physics paramters are also entered

In [1]:
# Use the KS framework for calculations
from models import DefaultModel
import UtilityFunctions as  uf
import UsefulInputs as ui

dm = DefaultModel("../../amplitude_calculations/output/KS_default.pickle",
                 binning_file="../../python/input/KsPiPi_optimal.pickle")

using defualt (trivial) efficiency


In [2]:
# Use Fi, ci, si also used in toys to compare results
import numpy as np
np.set_printoptions(precision=3, suppress=True)

use_KsKK = 0
if use_KsKK:
    dm.set_Fi(ui.run2_Fi_DD_KsKK) # from the LHCb measurement input
    dm.set_ci_si(ui.cleo_ci_KsKK, ui.cleo_si_KsKK)
else:
    dm.set_Fi(ui.run2_Fi_DD) # from the LHCb measurement input
    dm.set_ci_si(ui.cleo_ci, ui.cleo_si)
    
physics_param = [uf.deg_to_rad(75.), 0.1, uf.deg_to_rad(140.)]
xy = uf.get_xy(physics_param) # calculate x, y, N for these physics param

Nplus = 1000
Nminus = 1000

## Python definitions
This section just defines the functions above in python and can be skipped

In [3]:
def calc_yields(xyF):
    xy = xyF[0:4]
    F = xyF[4:]
    old_F = dm.Fi
    dm.set_Fi(F)
    yields = dm.predict_yields(xy, Nplus, Nminus)
    dm.set_Fi(old_F)
    return yields

def calc_Y(xyF):
    xy = xyF[0:4]
    F = xyF[4:]
    old_F = dm.Fi
    dm.set_Fi(F)
    Y = dm.predict_yields(xy, 1, 1, normalize=False)
    dm.set_Fi(old_F)
    return Y

def calc_chi2(xyF, data):
    xy = xyF[0:4]
    F = xyF[4:]
    old_F = dm.Fi
    dm.set_Fi(F)
    chi2 = dm.yields_chi_square(xy, data)
    dm.set_Fi(old_F)
    return chi2

def num_grad_Y(xyF, i, delta=1e-5):
    delta_xyF = 0 * xyF
    delta_xyF[i] = delta
    y1 = calc_Y(xyF+delta_xyF)
    y2 = calc_Y(xyF-delta_xyF)
    return (y1 - y2)/(2*delta)

def ana_grad_Y(xyF, i):
    xy = xyF[0:4]
    F = xyF[4:]
    F_inv = F[-1::-1]
    Ni = calc_yields(xyF) # just to get bin number ...
    N=len(Ni)/2
    grad = 0*Ni
    if i == 0: # xm
        grad[N:] = 2*xy[i]*F_inv+2*np.sqrt(F*F_inv)*dm.ci
    if i == 1: # ym
        grad[N:] = 2*xy[i]*F_inv+2*np.sqrt(F*F_inv)*dm.si
    if i == 2: # xp
        grad[0:N] = 2*xy[i]*F+2*np.sqrt(F*F_inv)*dm.ci
    if i == 3: # yp
        grad[0:N] = 2*xy[i]*F-2*np.sqrt(F*F_inv)*dm.si
    if i >= 4: # one of the Fi
        bin_i = i - 4
        bin_i_inv = N-1-bin_i
        # N+ i contribution
        grad[bin_i] = xy[2]**2+xy[3]**2+F_inv[bin_i]*(dm.ci[bin_i]*xy[2]-dm.si[bin_i]*xy[3])/np.sqrt(F[bin_i]*F_inv[bin_i])
        # N- i contribution
        grad[bin_i+N] = 1+F_inv[bin_i]*(dm.ci[bin_i]*xy[0]+dm.si[bin_i]*xy[1])/np.sqrt(F[bin_i]*F_inv[bin_i])
        # N+ -i contribution
        grad[bin_i_inv] = 1+F_inv[bin_i]*(dm.ci[bin_i_inv]*xy[2]-dm.si[bin_i_inv]*xy[3])/np.sqrt(F[bin_i]*F_inv[bin_i])
        # N- -i contribution
        grad[bin_i_inv+N] = xy[0]**2+xy[1]**2+F_inv[bin_i]*(dm.ci[bin_i_inv]*xy[0]+dm.si[bin_i_inv]*xy[1])/np.sqrt(F[bin_i]*F_inv[bin_i])
    return grad

def comp_grad_Y(i):
    print "N_gY_{}".format(i), num_grad_Y(np.concatenate([xy, dm.Fi]), i)
    print "A_gY_{}".format(i), ana_grad_Y(np.concatenate([xy, dm.Fi]), i)
    print "-"

test = False
if test:
    yields = dm.predict_yields(xy, 1000, 1000)
    for i in range(4+len(yields)/2):
        comp_grad_Y(i)

In [4]:
def num_grad_yields(xyF, i, delta=1e-5):
    delta_xyF = 0 * xyF
    delta_xyF[i] = delta
    y1 = calc_yields(xyF+delta_xyF)
    y2 = calc_yields(xyF-delta_xyF)
    return (y1 - y2)/(2*delta)

def ana_grad_yields(xyF, i):
    Ni = calc_yields(xyF) # just to get bin number ...
    N=len(Ni)/2
    hp = sum(Ni[:N])
    hm = sum(Ni[N:])
    Y = calc_Y(xyF)
    dY = ana_grad_Y(xyF, i)
    Yp = Y[:N]; 
    Ym = Y[N:]
    dYp = dY[:N]; 
    dYm = dY[N:]
    grad_p = hp/sum(Yp)*(dYp-Yp*sum(dYp)/sum(Yp))
    grad_m = hm/sum(Ym)*(dYm-Ym*sum(dYm)/sum(Ym))
    return np.concatenate([grad_p, grad_m])

def comp_grad_N(i):
    print "N_gN_{}".format(i), num_grad_yields(np.concatenate([xy, dm.Fi]), i)
    print "A_gN_{}".format(i), ana_grad_yields(np.concatenate([xy, dm.Fi]), i)
    print "-"
test = False
if test:
    for i in range(4+len(yields)/2):
        comp_grad_N(i)


In [5]:
def num_grad_chi2(xyF, i, data, delta=1e-5):
    delta_xyF = 0 * xyF
    delta_xyF[i] = delta
    y1 = calc_chi2(xyF+delta_xyF, data)
    y2 = calc_chi2(xyF-delta_xyF, data)
    return (y1 - y2)/(2*delta)

def ana_grad_chi2(xyF, i, data):
    Nhat = calc_yields(xyF)
    dNhat = ana_grad_yields(xyF, i)
    grad_chi2 = sum( (Nhat-data)/data*dNhat)
    return grad_chi2

def comp_grad_chi2(xyF, i):
    print "N_gChi2_{}".format(i), num_grad_chi2(xyF, i, yields)
    print "A_gChi2_{}".format(i), ana_grad_chi2(xyF, i, yields)
    print "-"

test = False
if test:
    for i in range(4+len(yields)/2):
        print "Exact sol"
        comp_grad_chi2(np.concatenate([xy, dm.Fi]), i)
        print "Crappy Fi"
    #     comp_grad_chi2(np.concatenate([xy, [0.25, 0.25, 0.25, 0.25]]), i)

In [6]:
def num_hess_yields(xyF, i, j, delta=1e-5):
    delta_xyF = 0 * xyF
    delta_xyF[j] = delta
    y1 = num_grad_yields(xyF+delta_xyF, i)
    y2 = num_grad_yields(xyF-delta_xyF, i)
    return (y1 - y2)/(2*delta)

def num_hess_chi2(xyF, i, j, data, delta=1e-5):
    delta_xyF = 0 * xyF
    delta_xyF[j] = delta
    y1 = num_grad_chi2(xyF+delta_xyF, i, data)
    y2 = num_grad_chi2(xyF-delta_xyF, i, data)
    return (y1 - y2)/(2*delta)

def ana_simple_hess_chi2(xyF, i, j): # Does not take data, as it is a proximation close to an optimal solution
    N = calc_yields(xyF)
    dN_di = ana_grad_yields(xyF, i)
    dN_dj = ana_grad_yields(xyF, j)
    return sum(dN_di*dN_dj/N)

def comp_hess_chi2(xyF, i, j, data):
    print "N_hess_Chi2_{}_{}       ".format(i, j), num_hess_chi2(xyF, i, j, data)
    print "A_simple_hess_Chi2_{}_{}".format(i, j), ana_simple_hess_chi2(xyF, i, j)
    print "-"
    
test = False
if test:
    for i in range(4+len(yields)/2):
        for j in range(4+len(yields)/2):
            comp_hess_chi2(np.concatenate([xy, dm.Fi]), i, j, yields)
    #         comp_hess_chi2(np.concatenate([xy, [0.25, 0.25, 0.25, 0.25]]), i, j, yields)



In [7]:
# a few utility functions 
def get_V(H):
    return np.linalg.inv(H)

def get_uncertainties(H):
    V = get_V(H)
    return np.sqrt(np.diag(V))

def get_correlations(H):
    unc = get_uncertainties(H)
    V = get_V(H)
    diag_mat = np.diag(1/unc)
    corr = diag_mat.dot(V).dot(np.transpose(diag_mat))
    return corr

## Looking at the calculation results
We can now calculate the Hession matrix

In [8]:
# Start constructing the Hessian
H = np.zeros((4+len(dm.Fi), 4+len(dm.Fi))) # 4 xy, 16 Fi's
H_num = H
for i in range(4+len(dm.Fi)):
    for j in range(4+len(dm.Fi)):
        H[i, j] = ana_simple_hess_chi2(np.concatenate([xy, dm.Fi]), i, j)

### Simple case: only floating x and y
When only floating $x$ and $y$ parameters, the hessian matrix is a 2x2 2x2 block matrix, as the equations decouple completely:

In [9]:
Hxy = H[0:4,0:4]
print Hxy

[[1673.465  363.372    0.       0.   ]
 [ 363.372 1478.303    0.       0.   ]
 [   0.       0.    2139.898 -331.005]
 [   0.       0.    -331.005 3003.441]]


This matrix can be inverted to get the covariance matrix, from which the expected uncertainties and correlations can be obtained

In [10]:
print "Uncertainties (xm, ym, xp, yp)"
print get_uncertainties(Hxy)
print "\nCorrelations:"
print get_correlations(Hxy)

Uncertainties (xm, ym, xp, yp)
[0.025 0.027 0.022 0.018]

Correlations:
[[ 1.    -0.231  0.     0.   ]
 [-0.231  1.     0.     0.   ]
 [ 0.     0.     1.     0.131]
 [ 0.     0.     0.131  1.   ]]


This can for example be compare to a high stats signal only toy study with fixed Ki, one specific relisation of which gave

$$\rho(x_-, y_-) = -0.217 \qquad \rho(x_+, y_+)=+0.167$$

which certainly agrees very well with above (input $rB=0.1$, $\gamma=75^\circ$, $\delta_B=140^\circ$ above is necessary for the two numbers to match ...).

### What happens if the $F_i$ are freed?
In that case, we include the partial derivatives of all but the last $F_i$ in the Hessian that we invert (corresponsing to the last $F_i$ being fixed, to remove an arbitrary normalisation)

In [11]:
HF = H[0:-1,0:-1]
print "Uncertainties (xm, ym, xp, yp)"
print get_uncertainties(HF)[0:4]
print "\nCorrelations:"
print get_correlations(HF)[0:4,0:4]

Uncertainties (xm, ym, xp, yp)
[0.091 0.107 0.091 0.082]

Correlations:
[[1.    0.487 0.922 0.633]
 [0.487 1.    0.601 0.928]
 [0.922 0.601 1.    0.725]
 [0.633 0.928 0.725 1.   ]]


Now the uncertainties on x and y increased by a factor 4! Furthermore, very large correlations are now exhibited between $x_+$ and $x_-$ (same for $y$), due to the equations beeing heavily coupled by the unconstrained $F_i$. 

This behaviour is even worse for small $r_B$, as can be seen by changing the input value above.

Below, we print correlations in the same order as the toy setups, for easy comparison

In [12]:
corr = get_correlations(HF)[0:4,0:4]

print "{:1.3f} {:1.3f} {:1.3f} {:1.3f}".format(corr[0,0], corr[0,2], corr[0,1], corr[0,3])
print " "*6+"{:1.3f} {:1.3f} {:1.3f}".format(corr[2,2],  corr[2,1], corr[2,3])
print " "*12+"{:1.3f} {:1.3f}".format(  corr[1,1], corr[1,3])
print " "*18+"{:1.3f} ".format(corr[3,3])

1.000 0.922 0.487 0.633
      1.000 0.601 0.725
            1.000 0.928
                  1.000 


## Adding constraints to the $F_i$
One can use this framework to examine the effect of constraints on $F_i$ by extending the likelihood function and adding a term to the Hessian elements involving $F_i$. I haven't done so, however.

# A test of autograd versus minimize Hessians
We can use these calculations to compare
* the analytical Hessian
* the Hessian returned by the dm.fit call (res.hess_inv)
* the Hessian as calculated by `autograd`, an automatical differentiation library

In [13]:
H1 = Hxy
print H1

[[1673.465  363.372    0.       0.   ]
 [ 363.372 1478.303    0.       0.   ]
 [   0.       0.    2139.898 -331.005]
 [   0.       0.    -331.005 3003.441]]


In [14]:
n = dm.predict_yields(xy, Nplus, Nminus)
res = dm.fit(n)
H2 = np.linalg.inv(res.hess_inv)
print(H2)

Fitting data using DefaultModel()
 Succes: True
 Msg   : Optimization terminated successfully.
 Fun   : 2.58502599518e-13
 p-val : 1.0
[[1645.209  355.038  -14.616   11.847]
 [ 355.038 1436.128  -21.162   20.198]
 [ -14.616  -21.162 2110.308 -303.701]
 [  11.847   20.198 -303.701 2978.665]]


In [16]:
from autograd import hessian
H = hessian(dm.yields_chi_square)
H3 = H(res.x, n)
print H3

[[1673.465  363.372    0.       0.   ]
 [ 363.372 1478.302    0.       0.   ]
 [   0.       0.    2139.898 -331.006]
 [   0.       0.    -331.006 3003.441]]


As can be seen, `autograd` indeed recovers the **exact** Hessian, without the numerical effects at play in the `minimize` version in `dm.fit()`. Therefore, now every fit result has the Hessian, covariance, and correlation matrix attached to it using autograd, before being returned in `DefaultModel::fit`.